# This notebook will use GEE using packages geemap, ee, and scikit-learn to train a regression model

### The study area is a city in Pakistan called Charsadda which is in the norhtern part of the country. It is chosen because it has a variation of elevation and greenary in its surroundings.

### Initialize GEE

In [2]:
import ee
import geemap

In [3]:
# # If you are running this notebook for the first time, you need to activate the command below for the authentication flow:
# ee.Authenticate()

In [4]:
try:
    # Initialize the library.
    ee.Initialize()
    print('Google Earth Engine has initialized successfully!')
except ee.EEException as e:
    print('Google Earth Engine has failed to initialize!')
except:
    print("Unexpected error:", sys.exc_info()[0])
    raise

Google Earth Engine has initialized successfully!


### Define study area and dates

In [5]:
# A point over Charsadda, Pakistan
point = ee.Geometry.Point([71.755005, 34.16239])

# Define the start and end dates
start_date = '2020-01-01'
end_date = '2020-12-31'

In [6]:
# Initialize the map
Map = geemap.Map()

In [7]:
# Visualize the Map centered over the area
Map.centerObject(point, 14)
Map.addLayer(point)
Map

Map(center=[34.16239, 71.755005], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(…

### Calculate NDVI and generate monthly composites

In [8]:
# Function to calculate NDVI from input bands
def calculate_ndvi(image):
    ndvi = image.normalizedDifference(['SR_B5', 'SR_B4']).rename('NDVI')
    return image.addBands(ndvi)

In [52]:
# Select the images from GEE using the input location and dates
image_collection = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2') \
                      .filterBounds(point) \
                      .filterDate(start_date, end_date) \
                      .map(calculate_ndvi)
                      # .filter(ee.Filter.lt('CLOUD_COVER', 10))
                      

In [53]:
# Quickly check the first image
image_collection.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'SR_B1',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7561, 7701],
   'crs': 'EPSG:32642',
   'crs_transform': [30, 0, 638685, 0, -30, 3948315]},
  {'id': 'SR_B2',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7561, 7701],
   'crs': 'EPSG:32642',
   'crs_transform': [30, 0, 638685, 0, -30, 3948315]},
  {'id': 'SR_B3',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7561, 7701],
   'crs': 'EPSG:32642',
   'crs_transform': [30, 0, 638685, 0, -30, 3948315]},
  {'id': 'SR_B4',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
    'min': 0,
    'max': 65535},
   'dimensions': [7561, 7701],
   'crs': 'EPSG:32642',
   'crs_transform': [30, 0, 638685, 0, -30, 3948315]},
  {'id': 'SR_B5',
   'data_type': {'type': 'PixelType',
    'precision': 'int',
 

In [54]:
# Function to calculate monthly mean from all images
def monthly_median_ndvi(year, month):
    start = ee.Date.fromYMD(year, month, 1)
    end = start.advance(1, 'month')
    
    # Filter images in the given month and calculate the median NDVI
    monthly_ndvi = image_collection.filterDate(start, end).select('NDVI').median()
    
    return monthly_ndvi.set('system:time_start', start.millis())

In [55]:
# Generate monthly composites
monthly_ndvi_composites = ee.ImageCollection([monthly_median_ndvi(2020, month) for month in range(1, 13)])
monthly_ndvi_composites

In [56]:
# Visualize the monthyl NDVIs on map
ndvi_vis_params = {
    'min': 0.0,
    'max': 1.0,
    'palette': ['blue', 'white', 'green']
}

# Add each monthly NDVI composite to the map
for month in range(1, 13):
    ndvi_image = monthly_ndvi_composites.filter(ee.Filter.calendarRange(month, month, 'month')).first()
    Map.addLayer(ndvi_image, ndvi_vis_params, f'Month {month} - NDVI Median Composite')

In [57]:
Map

Map(bottom=1673508.0, center=[34.16239, 71.755005], controls=(WidgetControl(options=['position', 'transparent_…

In [58]:
# Compute the region boundary, in this case the bounds of the landsat image
region = image_collection.first().geometry().bounds()
region.coordinates().getInfo()

[[[70.52236131279385, 33.55393964230097],
  [72.97562017178673, 33.55393964230097],
  [72.97562017178673, 35.656882688566846],
  [70.52236131279385, 35.656882688566846],
  [70.52236131279385, 33.55393964230097]]]

In [59]:
# Add it to the Map
Map.addLayer(region)

### Get the elevation and precipitation datasets

In [60]:
# Select the elevation dataset
elevation = ee.Image('USGS/SRTMGL1_003').select('elevation')

Map.addLayer(elevation, {min: 10, max: 5000}, 'Elevation')

In [61]:

precip = (ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY')
                  .filter(ee.Filter.date(start_date, end_date))).select('precipitation')
precip.first().getInfo()

{'type': 'Image',
 'bands': [{'id': 'precipitation',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'dimensions': [7200, 2000],
   'crs': 'EPSG:4326',
   'crs_transform': [0.05, 0, -180, 0, -0.05, 50]}],
 'version': 1710264987660386,
 'id': 'UCSB-CHG/CHIRPS/DAILY/20200101',
 'properties': {'system:time_start': 1577836800000,
  'system:footprint': {'type': 'LinearRing',
   'coordinates': [[-180, -90],
    [180, -90],
    [180, 90],
    [-180, 90],
    [-180, -90]]},
  'system:time_end': 1577923200000,
  'system:asset_size': 4668072,
  'system:index': '20200101'}}

In [62]:
# Select the precipitation filtered by the dates
precipitation = ee.ImageCollection('UCSB-CHG/CHIRPS/DAILY').filterBounds(point).filterDate(start_date, end_date)

In [63]:
# Compute monthly precipitation from the dataset
def monthly_precipitation(year, month):
    start = ee.Date.fromYMD(year, month, 1)
    end = start.advance(1, 'month')
    
    # Filter images within the month and sum the daily precipitation
    monthly_precip = precipitation.filterDate(start, end).sum().rename('monthly_precipitation')
    return monthly_precip.set('system:time_start', start.millis())

In [64]:
# Generate monthly precipitation composites
monthly_precipitation_composites = ee.ImageCollection([monthly_precipitation(2020, month) for month in range(1, 13)])

In [65]:
# Add the precipitation data on the map
precip_vis_params = {
    'min': 0.0,
    'max': 300.0, 
    'palette': ['blue', 'cyan', 'yellow', 'orange', 'red']
}

# Add each monthly precipitation layer to the map
for month in range(1, 13):
    precip_image = monthly_precipitation_composites.filter(ee.Filter.calendarRange(month, month, 'month')).first()
    Map.addLayer(precip_image, precip_vis_params, f'Month {month} - Precipitation')

Map

Map(bottom=1673508.0, center=[34.16239, 71.755005], controls=(WidgetControl(options=['position', 'transparent_…

### Prepare dataset for sampling and modelling

In [66]:
# Generate annual NDVI so we have one NDVI for precipitaion of 23 months and elevation as predictor variables
annual_median_ndvi = monthly_ndvi_composites.median()
annual_median_ndvi

In [68]:
# Stack the monthly precipitation to bands
precip_stack = monthly_precipitation_composites.toBands()
precip_stack

In [69]:
# Add elevation to the stack
precip_el_stack = precip_stack.addBands(elevation.rename('elevation'))

# Add the NDVI to the stack
combined_stack = precip_el_stack.addBands(annual_median_ndvi.rename('NDVI'))

In [71]:
# Generate training samples
training = combined_stack.sample(
    **{
        'region': region,
        "scale": 30,
        "numPixels": 5000,
        "seed": 0,
        "geometries": True,  # Set this to False to ignore geometries
    }
)

Map.addLayer(training, {}, "training", False)
Map

Map(bottom=1673508.0, center=[34.16239, 71.755005], controls=(WidgetControl(options=['position', 'transparent_…

In [72]:
training.first().getInfo()

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [71.747410492735, 34.37488529121087]},
 'id': '0',
 'properties': {'0_monthly_precipitation': 95.34018754959106,
  '10_monthly_precipitation': 29.142422676086426,
  '11_monthly_precipitation': 21.244751930236816,
  '1_monthly_precipitation': 100.98373746871948,
  '2_monthly_precipitation': 170.350905418396,
  '3_monthly_precipitation': 206.16515469551086,
  '4_monthly_precipitation': 60.28784454095876,
  '5_monthly_precipitation': 21.725277423858643,
  '6_monthly_precipitation': 23.60567331314087,
  '7_monthly_precipitation': 58.27423572540283,
  '8_monthly_precipitation': 57.78390955924988,
  '9_monthly_precipitation': 7.722382601350546,
  'NDVI': 0.1581745147705078,
  'elevation': 411}}

In [73]:
# Convert the training samples to a pandas dataframe
import pandas as pd

In [74]:
data = training.getInfo()['features']
data_dict = [feature['properties'] for feature in data]
df = pd.DataFrame(data_dict)

In [75]:
df

,0_monthly_precipitation,10_monthly_precipitation,11_monthly_precipitation,1_monthly_precipitation,2_monthly_precipitation,3_monthly_precipitation,4_monthly_precipitation,5_monthly_precipitation,6_monthly_precipitation,7_monthly_precipitation,8_monthly_precipitation,9_monthly_precipitation,NDVI,elevation
0,95.340188,29.142423,21.244752,100.983737,170.350905,206.165155,60.287845,21.725277,23.605673,58.274236,57.783910,7.722383,0.158175,411
1,141.011530,83.918355,38.706804,83.532030,278.383991,282.350147,83.759596,38.087345,258.087039,85.726021,134.248535,23.875978,0.227769,1442
2,67.492120,49.198615,25.696477,47.121884,146.260679,198.420641,112.644975,15.666552,91.514676,19.766538,104.969462,13.305455,0.128447,1386
3,68.950802,44.395090,24.308187,33.551166,143.458541,228.143675,75.967534,53.084773,114.469245,100.014429,256.994081,5.290262,0.061046,2669
4,90.338189,61.674423,39.830410,66.271455,243.979823,239.980920,60.655208,39.640158,179.491253,100.986557,145.071060,25.419806,0.196031,1199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3522,81.167246,49.301723,14.779800,58.073355,144.287837,177.248507,39.085194,17.654768,15.573502,22.465552,54.138325,9.037371,0.216952,1791
3523,70.060311,35.484322,15.276512,64.897587,167.302860,154.370913,50.373299,15.136239,11.198898,37.632156,26.782185,7.113817,0.114563,417
3524,64.398317,46.986526,36.254935,59.731375,184.551050,124.171856,66.089892,35.171061,42.888556,92.685954,71.007451,2.581240,0.141103,1088
3525,65.586147,60.689449,37.937573,38.748876,177.838769,138.086676,75.174569,43.696834,65.445966,132.273859,103.139542,9.166960,0.247359,2067


In [76]:
# Save the data to a csv file
df.to_csv('Training.csv', index=False)

### Train the model, in this case regression model

In [32]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [77]:
# Drop any null values
df = df.dropna(subset=['NDVI', 'elevation', '0_monthly_precipitation', '10_monthly_precipitation', '11_monthly_precipitation', '1_monthly_precipitation', '2_monthly_precipitation', '3_monthly_precipitation', '4_monthly_precipitation', '5_monthly_precipitation', '6_monthly_precipitation', '7_monthly_precipitation', '8_monthly_precipitation', '9_monthly_precipitation'])

In [78]:
# Define predictor and target variables for training
X = df[['elevation', '0_monthly_precipitation', '10_monthly_precipitation', '11_monthly_precipitation', '1_monthly_precipitation', '2_monthly_precipitation', '3_monthly_precipitation', '4_monthly_precipitation', '5_monthly_precipitation', '6_monthly_precipitation', '7_monthly_precipitation', '8_monthly_precipitation', '9_monthly_precipitation']]  # Features
y = df['NDVI']

In [79]:
# Split the dataset into a ratio of 70/30 for training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [80]:
# Train the model
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [81]:
# Make predictions
y_pred = model.predict(X_test)

In [ ]:
# Compute 